# 15 - SIFT + SfM Iterative Test Localization (Submission)

This notebook is based on the core method from notebook 12 **before** the feature-focus sweep.

Goal:
1. Build train-test batches/segments from IDs.
2. Estimate relative motion iteratively with SIFT + affine/SfM cues.
3. Use optional segment closure to next train anchor to reduce drift.
4. Export `submission.csv` for all test frames.


In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

plt.rcParams['figure.dpi'] = 130
np.set_printoptions(precision=4, suppress=True)


In [ ]:
# Basic checks + paths + config
if not hasattr(cv2, 'SIFT_create'):
    raise RuntimeError('OpenCV build has no SIFT. Install opencv-contrib-python.')

CANDIDATE_ROOTS = [Path.cwd(), Path.cwd().parent, Path('.'), Path('..')]
PROJECT_ROOT = None
_seen = set()
for cand in CANDIDATE_ROOTS:
    try:
        root = cand.resolve()
    except Exception:
        continue
    key = str(root)
    if key in _seen:
        continue
    _seen.add(key)
    if (root / 'data' / 'data').exists():
        PROJECT_ROOT = root
        break
if PROJECT_ROOT is None:
    raise FileNotFoundError('Could not find project root containing data/data.')

DATA_ROOT = PROJECT_ROOT / 'data' / 'data'
TRAIN_IMG_DIR = DATA_ROOT / 'train_data' / 'train_images'
TEST_IMG_DIR = DATA_ROOT / 'test_data' / 'test_images'
TRAIN_POS_CSV = DATA_ROOT / 'train_data' / 'train_pos.csv'
TRAIN_CAM_CSV = DATA_ROOT / 'train_data' / 'train_cam.csv'
TEST_CAM_CSV = DATA_ROOT / 'test_data' / 'test_cam.csv'
MAP_PATH = DATA_ROOT / 'map.png'

# Runtime
IMAGE_MAX_SIDE = 1000
PREPROCESS_MODE = 'gray_clahe'  # ['gray', 'gray_clahe', 'gray_denoise_clahe']

# SIFT settings (notebook 12 style: balanced)
SIFT_CFG = {
    'nfeatures': 6500,
    'contrast_thr': 0.03,
    'edge_thr': 12,
    'sigma': 1.6,
    'ratio_thr': 0.78,
}

# Geometry
AFFINE_RANSAC_THR = 3.0
HOMOGRAPHY_RANSAC_THR = 4.0
ESSENTIAL_RANSAC_THR = 1.5
MIN_MATCHES_FOR_GEOM = 8
MIN_AFFINE_INLIERS_FOR_MODEL = 10

# Batch/segment behavior
AUTO_SWITCH_TO_NEXT_ANCHOR = True
ANCHOR_SWITCH_MIN_INLIERS = 16
CLOSURE_MIN_INLIERS = 12

# Calibration behavior
CALIB_USE_BIDIRECTIONAL = True
CALIB_MAX_PAIRS = 260  # speed/quality tradeoff
ROBUST_ITERS = 5
HUBER_DELTA = 35.0
RIDGE = 1e-4

# Output
SUBMISSION_PATH = PROJECT_ROOT / 'build' / 'submission.csv'
ALT_SUBMISSION_PATH = PROJECT_ROOT / 'build' / 'submission_15_sift_sfm.csv'

print('project_root:', PROJECT_ROOT)
print('train_img_dir:', TRAIN_IMG_DIR)
print('test_img_dir:', TEST_IMG_DIR)


In [ ]:
# Load metadata
train_pos_df = pd.read_csv(TRAIN_POS_CSV)
train_cam_df = pd.read_csv(TRAIN_CAM_CSV)
test_cam_df = pd.read_csv(TEST_CAM_CSV)

for c in ['id', 'x_pixel', 'y_pixel']:
    if c not in train_pos_df.columns:
        raise KeyError(f'Missing column in train_pos.csv: {c}')
for c in ['id', 'fx', 'fy', 'cx', 'cy']:
    if c not in train_cam_df.columns:
        raise KeyError(f'Missing column in train_cam.csv: {c}')
    if c not in test_cam_df.columns:
        raise KeyError(f'Missing column in test_cam.csv: {c}')

train_df = train_cam_df.merge(train_pos_df, on='id', how='inner').copy()
train_df['id'] = train_df['id'].astype(int)
train_df = train_df.sort_values('id').reset_index(drop=True)

train_cam_df = train_cam_df.copy()
train_cam_df['id'] = train_cam_df['id'].astype(int)
test_cam_df = test_cam_df.copy()
test_cam_df['id'] = test_cam_df['id'].astype(int)

train_ids = sorted(train_df['id'].astype(int).tolist())
test_ids = sorted(test_cam_df['id'].astype(int).tolist())

train_pos_map = {
    int(r['id']): np.array([float(r['x_pixel']), float(r['y_pixel'])], dtype=np.float64)
    for _, r in train_df.iterrows()
}
train_cam_map = {int(r['id']): r for _, r in train_cam_df.iterrows()}
test_cam_map = {int(r['id']): r for _, r in test_cam_df.iterrows()}

train_id_set = set(train_ids)
test_id_set = set(test_ids)


def build_test_segments(ids: List[int]) -> List[Tuple[int, int]]:
    if len(ids) == 0:
        return []
    segs: List[Tuple[int, int]] = []
    s = ids[0]
    p = ids[0]
    for x in ids[1:]:
        if x == p + 1:
            p = x
        else:
            segs.append((s, p))
            s = x
            p = x
    segs.append((s, p))
    return segs


test_segments = build_test_segments(test_ids)

seg_rows = []
for i, (s, e) in enumerate(test_segments):
    prev_train = max([t for t in train_ids if t < s], default=None)
    next_train = min([t for t in train_ids if t > e], default=None)
    seg_rows.append({
        'segment_idx': i,
        'start_id': s,
        'end_id': e,
        'length': e - s + 1,
        'prev_train': prev_train,
        'next_train': next_train,
    })

segments_df = pd.DataFrame(seg_rows)
print('train frames:', len(train_ids), 'test frames:', len(test_ids), 'test segments:', len(test_segments))
display(segments_df)


In [ ]:
# Utilities: image loading, preprocessing, intrinsics
_IMG_CACHE: Dict[Tuple[int, Optional[int]], Tuple[np.ndarray, float]] = {}
_PREPROC_CACHE: Dict[Tuple[int, Optional[int], str], Tuple[np.ndarray, float]] = {}
_FEATURE_CACHE: Dict[Tuple[int, Optional[int], str, Tuple], Tuple[List[cv2.KeyPoint], Optional[np.ndarray], np.ndarray, np.ndarray, float]] = {}


def infer_source(image_id: int) -> str:
    if int(image_id) in train_id_set:
        return 'train'
    if int(image_id) in test_id_set:
        return 'test'
    raise KeyError(f'Image id {image_id} in neither train nor test metadata.')


def resolve_image_path(image_id: int) -> Path:
    src = infer_source(int(image_id))
    base_dir = TRAIN_IMG_DIR if src == 'train' else TEST_IMG_DIR
    stems = [f'{int(image_id):04d}', str(int(image_id))]
    exts = ['.JPG', '.jpg', '.jpeg', '.JPEG', '.png', '.PNG']
    for st in stems:
        for ext in exts:
            p = base_dir / f'{st}{ext}'
            if p.exists():
                return p
    raise FileNotFoundError(f'Image not found for id={image_id} in {base_dir}')


def load_image_rgb(image_id: int, max_side: Optional[int]) -> Tuple[np.ndarray, float]:
    key = (int(image_id), max_side)
    if key in _IMG_CACHE:
        return _IMG_CACHE[key]

    p = resolve_image_path(int(image_id))
    bgr = cv2.imread(str(p), cv2.IMREAD_COLOR)
    if bgr is None:
        raise RuntimeError(f'Cannot read image: {p}')
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    scale = 1.0
    if max_side is not None:
        h, w = rgb.shape[:2]
        m = max(h, w)
        if m > int(max_side):
            scale = float(max_side) / float(m)
            nw = max(32, int(round(w * scale)))
            nh = max(32, int(round(h * scale)))
            rgb = cv2.resize(rgb, (nw, nh), interpolation=cv2.INTER_AREA)

    _IMG_CACHE[key] = (rgb, scale)
    return rgb, scale


def preprocess_for_sift(img_rgb: np.ndarray, mode: str) -> np.ndarray:
    g = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    if mode == 'gray':
        return g
    if mode == 'gray_clahe':
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        return clahe.apply(g)
    if mode == 'gray_denoise_clahe':
        g2 = cv2.bilateralFilter(g, d=7, sigmaColor=45, sigmaSpace=45)
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        return clahe.apply(g2)
    raise KeyError(f'Unknown preprocess mode: {mode}')


def load_preprocessed_gray(image_id: int, max_side: Optional[int], mode: str) -> Tuple[np.ndarray, float]:
    key = (int(image_id), max_side, str(mode))
    if key in _PREPROC_CACHE:
        return _PREPROC_CACHE[key]
    rgb, scale = load_image_rgb(int(image_id), max_side=max_side)
    gray = preprocess_for_sift(rgb, mode=mode)
    _PREPROC_CACHE[key] = (gray, scale)
    return gray, scale


def get_cam_row(image_id: int) -> pd.Series:
    iid = int(image_id)
    if iid in train_cam_map:
        return train_cam_map[iid]
    if iid in test_cam_map:
        return test_cam_map[iid]
    raise KeyError(f'No intrinsics row for id={image_id}')


def scaled_K(image_id: int, image_scale: float) -> np.ndarray:
    row = get_cam_row(int(image_id))
    fx = float(row['fx']) * float(image_scale)
    fy = float(row['fy']) * float(image_scale)
    cx = float(row['cx']) * float(image_scale)
    cy = float(row['cy']) * float(image_scale)
    return np.array([[fx, 0.0, cx], [0.0, fy, cy], [0.0, 0.0, 1.0]], dtype=np.float64)


In [ ]:
# Pair motion estimation (SIFT + affine + optional SfM)
@dataclass
class PairMotion:
    id0: int
    id1: int
    success: bool
    reason: str
    matches: int
    affine_inliers: int
    homography_inliers: int
    sfm_inliers: int
    affine_M_1to0: Optional[np.ndarray]
    H_1to0: Optional[np.ndarray]
    tx: float
    ty: float
    rot_deg: float
    scale: float
    sfm_t: Optional[np.ndarray]


def get_features_for_id(image_id: int, cfg: Dict[str, float]):
    cfg_key = (
        int(cfg['nfeatures']),
        float(cfg['contrast_thr']),
        float(cfg['edge_thr']),
        float(cfg['sigma']),
        str(PREPROCESS_MODE),
    )
    key = (int(image_id), IMAGE_MAX_SIDE, PREPROCESS_MODE, cfg_key)
    if key in _FEATURE_CACHE:
        return _FEATURE_CACHE[key]

    gray, scale = load_preprocessed_gray(int(image_id), IMAGE_MAX_SIDE, PREPROCESS_MODE)
    K = scaled_K(int(image_id), image_scale=scale)

    sift = cv2.SIFT_create(
        nfeatures=int(cfg['nfeatures']),
        contrastThreshold=float(cfg['contrast_thr']),
        edgeThreshold=float(cfg['edge_thr']),
        sigma=float(cfg['sigma']),
    )
    kps, desc = sift.detectAndCompute(gray, None)
    if kps is None:
        kps = []

    _FEATURE_CACHE[key] = (kps, desc, gray, K, scale)
    return _FEATURE_CACHE[key]


_PAIR_MOTION_CACHE: Dict[Tuple[int, int, Tuple], PairMotion] = {}


def estimate_pair_motion(id0: int, id1: int, cfg: Dict[str, float]) -> PairMotion:
    cfg_key = (
        int(cfg['nfeatures']),
        float(cfg['contrast_thr']),
        float(cfg['edge_thr']),
        float(cfg['sigma']),
        float(cfg['ratio_thr']),
    )
    cache_key = (int(id0), int(id1), cfg_key)
    if cache_key in _PAIR_MOTION_CACHE:
        return _PAIR_MOTION_CACHE[cache_key]

    k0, d0, g0, K0, _ = get_features_for_id(int(id0), cfg)
    k1, d1, g1, K1, _ = get_features_for_id(int(id1), cfg)

    if d0 is None or d1 is None or len(k0) < 2 or len(k1) < 2:
        out = PairMotion(int(id0), int(id1), False, 'no_descriptors', 0, 0, 0, 0, None, None, np.nan, np.nan, np.nan, np.nan, None)
        _PAIR_MOTION_CACHE[cache_key] = out
        return out

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    knn = bf.knnMatch(d0, d1, k=2)

    good = []
    ratio_thr = float(cfg['ratio_thr'])
    for pair in knn:
        if len(pair) < 2:
            continue
        m, n = pair
        if m.distance < ratio_thr * n.distance:
            good.append(m)

    if len(good) < MIN_MATCHES_FOR_GEOM:
        out = PairMotion(int(id0), int(id1), False, 'few_matches', len(good), 0, 0, 0, None, None, np.nan, np.nan, np.nan, np.nan, None)
        _PAIR_MOTION_CACHE[cache_key] = out
        return out

    pts0 = np.array([k0[m.queryIdx].pt for m in good], dtype=np.float32)
    pts1 = np.array([k1[m.trainIdx].pt for m in good], dtype=np.float32)

    M, inl_aff = cv2.estimateAffinePartial2D(
        pts1,
        pts0,
        method=cv2.RANSAC,
        ransacReprojThreshold=float(AFFINE_RANSAC_THR),
        maxIters=20000,
        confidence=0.999,
        refineIters=50,
    )
    if inl_aff is not None:
        inl_aff = inl_aff.ravel().astype(bool)
        n_inl_aff = int(inl_aff.sum())
    else:
        inl_aff = np.zeros((len(good),), dtype=bool)
        n_inl_aff = 0

    if M is not None:
        rot_deg = float(np.degrees(np.arctan2(M[1, 0], M[0, 0])))
        scale = float(np.sqrt(M[0, 0] ** 2 + M[1, 0] ** 2))
        tx = float(M[0, 2])
        ty = float(M[1, 2])
    else:
        rot_deg = np.nan
        scale = np.nan
        tx = np.nan
        ty = np.nan

    method = cv2.USAC_MAGSAC if hasattr(cv2, 'USAC_MAGSAC') else cv2.RANSAC
    H, inl_h = cv2.findHomography(
        pts1,
        pts0,
        method=method,
        ransacReprojThreshold=float(HOMOGRAPHY_RANSAC_THR),
        maxIters=20000,
        confidence=0.999,
    )
    if inl_h is not None:
        n_inl_h = int(inl_h.ravel().astype(bool).sum())
    else:
        n_inl_h = 0

    sfm_t = None
    n_inl_pose = 0
    if len(good) >= 8:
        pts0n = cv2.undistortPoints(pts0.reshape(-1, 1, 2), K0, None).reshape(-1, 2)
        pts1n = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K1, None).reshape(-1, 2)
        E, inl_e = cv2.findEssentialMat(
            pts0n,
            pts1n,
            cameraMatrix=np.eye(3),
            method=cv2.RANSAC,
            prob=0.999,
            threshold=float(ESSENTIAL_RANSAC_THR),
        )
        if E is not None and inl_e is not None and int(inl_e.sum()) >= 8:
            _, R, t, inl_pose = cv2.recoverPose(
                E,
                pts0n,
                pts1n,
                cameraMatrix=np.eye(3),
                mask=inl_e.astype(np.uint8).reshape(-1, 1),
            )
            sfm_t = t.reshape(3).astype(np.float64)
            if inl_pose is not None:
                n_inl_pose = int(inl_pose.ravel().astype(bool).sum())

    success = (M is not None) and (n_inl_aff >= MIN_AFFINE_INLIERS_FOR_MODEL)
    reason = 'ok' if success else 'weak_affine'

    out = PairMotion(
        id0=int(id0),
        id1=int(id1),
        success=bool(success),
        reason=reason,
        matches=int(len(good)),
        affine_inliers=int(n_inl_aff),
        homography_inliers=int(n_inl_h),
        sfm_inliers=int(n_inl_pose),
        affine_M_1to0=M,
        H_1to0=H,
        tx=float(tx),
        ty=float(ty),
        rot_deg=float(rot_deg),
        scale=float(scale),
        sfm_t=sfm_t,
    )
    _PAIR_MOTION_CACHE[cache_key] = out
    return out


def motion_to_feature_vector(m: PairMotion) -> np.ndarray:
    sfm_tx, sfm_ty, sfm_tz = 0.0, 0.0, 0.0
    if m.sfm_t is not None:
        t = m.sfm_t.astype(np.float64)
        tn = float(np.linalg.norm(t))
        if tn > 1e-12:
            t = t / tn
            sfm_tx, sfm_ty, sfm_tz = float(t[0]), float(t[1]), float(t[2])

    rot_rad = 0.0 if not np.isfinite(m.rot_deg) else float(np.deg2rad(m.rot_deg))
    scale_log = 0.0 if (not np.isfinite(m.scale) or m.scale <= 1e-8) else float(np.log(m.scale))
    tx = 0.0 if not np.isfinite(m.tx) else float(m.tx)
    ty = 0.0 if not np.isfinite(m.ty) else float(m.ty)

    return np.array([tx, ty, rot_rad, scale_log, sfm_tx, sfm_ty, sfm_tz], dtype=np.float64)


In [ ]:
# Robust linear mapping: motion features -> map delta
@dataclass
class MotionLinearModel:
    beta: np.ndarray  # shape (d+1, 2)
    feature_names: List[str]
    med_err: float
    mean_err: float
    used_pairs: int


def fit_robust_linear_model(
    X: np.ndarray,
    Y: np.ndarray,
    base_w: np.ndarray,
    robust_iters: int = 5,
    huber_delta: float = 35.0,
    ridge: float = 1e-4,
) -> np.ndarray:
    n, d = X.shape
    Xb = np.hstack([X, np.ones((n, 1), dtype=np.float64)])

    w = np.clip(base_w.astype(np.float64), 1e-6, None).copy()

    for _ in range(int(robust_iters)):
        W = np.sqrt(w).reshape(-1, 1)
        Xw = Xb * W
        Yw = Y * W

        XtX = Xw.T @ Xw
        XtY = Xw.T @ Yw
        XtX = XtX + float(ridge) * np.eye(XtX.shape[0], dtype=np.float64)

        beta = np.linalg.solve(XtX, XtY)

        pred = Xb @ beta
        err = np.linalg.norm(Y - pred, axis=1)
        hub = np.ones_like(err)
        bad = err > float(huber_delta)
        hub[bad] = float(huber_delta) / np.clip(err[bad], 1e-6, None)
        w = np.clip(base_w, 1e-6, None) * hub

    return beta


def predict_delta(model: MotionLinearModel, m: PairMotion) -> np.ndarray:
    x = motion_to_feature_vector(m)
    xb = np.concatenate([x, np.array([1.0], dtype=np.float64)], axis=0)
    y = xb @ model.beta
    return y.astype(np.float64)


# Build directed calibration pairs from consecutive train ids
calib_pairs: List[Tuple[int, int]] = []
for i in range(len(train_ids) - 1):
    a = int(train_ids[i])
    b = int(train_ids[i + 1])
    if b != a + 1:
        continue
    calib_pairs.append((a, b))
    if CALIB_USE_BIDIRECTIONAL:
        calib_pairs.append((b, a))

if CALIB_MAX_PAIRS is not None and len(calib_pairs) > int(CALIB_MAX_PAIRS):
    idx = np.linspace(0, len(calib_pairs) - 1, int(CALIB_MAX_PAIRS)).astype(int)
    calib_pairs = [calib_pairs[i] for i in idx]

rows = []
X_list = []
Y_list = []
W_list = []

for id0, id1 in calib_pairs:
    m = estimate_pair_motion(id0, id1, SIFT_CFG)
    gt = train_pos_map[id1] - train_pos_map[id0]

    rows.append({
        'id0': id0,
        'id1': id1,
        'matches': m.matches,
        'affine_inliers': m.affine_inliers,
        'sfm_inliers': m.sfm_inliers,
        'success': m.success,
        'gt_dx': float(gt[0]),
        'gt_dy': float(gt[1]),
        'tx': m.tx,
        'ty': m.ty,
        'rot_deg': m.rot_deg,
        'scale': m.scale,
    })

    if m.success:
        X_list.append(motion_to_feature_vector(m))
        Y_list.append(gt.astype(np.float64))
        W_list.append(max(1.0, float(m.affine_inliers)))

calib_df = pd.DataFrame(rows)
print('calibration directed pairs:', len(calib_pairs), 'usable for model:', len(X_list))
display(calib_df.head(20))

if len(X_list) < 20:
    raise RuntimeError('Too few usable calibration pairs. Try looser SIFT thresholds or lower MIN_AFFINE_INLIERS_FOR_MODEL.')

X = np.vstack(X_list)
Y = np.vstack(Y_list)
W = np.array(W_list, dtype=np.float64)

beta = fit_robust_linear_model(
    X=X,
    Y=Y,
    base_w=W,
    robust_iters=ROBUST_ITERS,
    huber_delta=HUBER_DELTA,
    ridge=RIDGE,
)

feature_names = ['tx', 'ty', 'rot_rad', 'log_scale', 'sfm_tx', 'sfm_ty', 'sfm_tz', 'bias']
model = MotionLinearModel(beta=beta, feature_names=feature_names, med_err=np.nan, mean_err=np.nan, used_pairs=int(len(X)))

pred_calib = np.hstack([X, np.ones((X.shape[0], 1), dtype=np.float64)]) @ beta
calib_err = np.linalg.norm(Y - pred_calib, axis=1)
model.med_err = float(np.median(calib_err))
model.mean_err = float(np.mean(calib_err))

coef_df = pd.DataFrame(
    model.beta,
    index=feature_names,
    columns=['dx_coef', 'dy_coef'],
)
print(f'calibration error: median={model.med_err:.2f}px mean={model.mean_err:.2f}px')
display(coef_df)

# fallback step from train GT consecutive deltas
train_step_deltas = []
for i in range(len(train_ids) - 1):
    a = int(train_ids[i])
    b = int(train_ids[i + 1])
    if b == a + 1:
        train_step_deltas.append(train_pos_map[b] - train_pos_map[a])
train_step_deltas = np.array(train_step_deltas, dtype=np.float64)
fallback_forward = np.median(train_step_deltas, axis=0)
fallback_backward = -fallback_forward
print('fallback_forward delta:', np.round(fallback_forward, 3))
print('fallback_backward delta:', np.round(fallback_backward, 3))


In [ ]:
# Build test batch plans
@dataclass
class SegmentPlan:
    segment_idx: int
    start_id: int
    end_id: int
    length: int
    anchor_train_id: int
    direction: str  # 'forward' or 'backward'
    ordered_test_ids: List[int]
    closing_train_id: Optional[int]
    prev_train_id: Optional[int]
    next_train_id: Optional[int]
    entry_inliers_forward: Optional[int]
    entry_inliers_backward: Optional[int]


def make_segment_plan(seg_idx: int, s: int, e: int) -> SegmentPlan:
    prev_train = max([t for t in train_ids if t < s], default=None)
    next_train = min([t for t in train_ids if t > e], default=None)

    # default rule from request
    if prev_train is None:
        # first test frames 1..12: anchor at 13 and run backward
        if next_train is None:
            raise RuntimeError(f'Segment {s}-{e} has no train anchor on either side.')
        anchor = int(next_train)
        direction = 'backward'
        ordered = list(range(int(e), int(s) - 1, -1))
        closing = prev_train
    else:
        anchor = int(prev_train)
        direction = 'forward'
        ordered = list(range(int(s), int(e) + 1))
        closing = next_train

    inl_fwd = None
    inl_bwd = None

    # optional rescue for hard segments: choose side with better entry match
    if AUTO_SWITCH_TO_NEXT_ANCHOR and (prev_train is not None) and (next_train is not None):
        m_fwd = estimate_pair_motion(int(prev_train), int(s), SIFT_CFG)
        m_bwd = estimate_pair_motion(int(next_train), int(e), SIFT_CFG)
        inl_fwd = int(m_fwd.affine_inliers)
        inl_bwd = int(m_bwd.affine_inliers)

        if (inl_fwd < ANCHOR_SWITCH_MIN_INLIERS) and (inl_bwd >= ANCHOR_SWITCH_MIN_INLIERS) and (inl_bwd > inl_fwd):
            anchor = int(next_train)
            direction = 'backward'
            ordered = list(range(int(e), int(s) - 1, -1))
            closing = prev_train

    return SegmentPlan(
        segment_idx=int(seg_idx),
        start_id=int(s),
        end_id=int(e),
        length=int(e - s + 1),
        anchor_train_id=int(anchor),
        direction=str(direction),
        ordered_test_ids=[int(x) for x in ordered],
        closing_train_id=int(closing) if closing is not None else None,
        prev_train_id=int(prev_train) if prev_train is not None else None,
        next_train_id=int(next_train) if next_train is not None else None,
        entry_inliers_forward=inl_fwd,
        entry_inliers_backward=inl_bwd,
    )


segment_plans: List[SegmentPlan] = []
for i, (s, e) in enumerate(test_segments):
    segment_plans.append(make_segment_plan(i, s, e))

plan_rows = []
for p in segment_plans:
    plan_rows.append({
        'segment_idx': p.segment_idx,
        'start_id': p.start_id,
        'end_id': p.end_id,
        'length': p.length,
        'anchor_train_id': p.anchor_train_id,
        'direction': p.direction,
        'closing_train_id': p.closing_train_id,
        'prev_train_id': p.prev_train_id,
        'next_train_id': p.next_train_id,
        'entry_inliers_forward': p.entry_inliers_forward,
        'entry_inliers_backward': p.entry_inliers_backward,
        'ordered_preview': str(p.ordered_test_ids[:6]) + (' ...' if len(p.ordered_test_ids) > 6 else ''),
    })
plan_df = pd.DataFrame(plan_rows)

display(plan_df)

# batch-step view (source frame -> target test frame)
batch_rows = []
for p in segment_plans:
    prev_id = int(p.anchor_train_id)
    prev_type = 'train'
    for step_idx, tid in enumerate(p.ordered_test_ids):
        batch_rows.append({
            'segment_idx': p.segment_idx,
            'step_idx': int(step_idx),
            'direction': p.direction,
            'source_id': int(prev_id),
            'source_type': prev_type,
            'target_id': int(tid),
            'target_type': 'test',
        })
        prev_id = int(tid)
        prev_type = 'test'
batch_steps_df = pd.DataFrame(batch_rows)

print('batch steps:', len(batch_steps_df))
display(batch_steps_df.head(40))


In [ ]:
# Iterative localization + optional closure correction

def choose_fallback_delta(direction: str) -> np.ndarray:
    if str(direction) == 'backward':
        return fallback_backward.copy()
    return fallback_forward.copy()


def predict_delta_with_fallback(m: PairMotion, direction: str, last_good_delta: Optional[np.ndarray]) -> Tuple[np.ndarray, str]:
    if m.success:
        return predict_delta(model, m), 'model'
    if last_good_delta is not None:
        return last_good_delta.copy(), 'last_good'
    return choose_fallback_delta(direction), 'fallback_median'


pred_map: Dict[int, np.ndarray] = {}
step_diag_rows: List[Dict[str, object]] = []
segment_diag_rows: List[Dict[str, object]] = []

for p in segment_plans:
    anchor_id = int(p.anchor_train_id)
    anchor_pos = train_pos_map[anchor_id].astype(np.float64).copy()

    segment_pred_local: Dict[int, np.ndarray] = {}
    prev_id = anchor_id
    prev_pos = anchor_pos.copy()
    last_good_delta = None

    for step_idx, tid in enumerate(p.ordered_test_ids):
        m = estimate_pair_motion(prev_id, tid, SIFT_CFG)
        dxy, mode = predict_delta_with_fallback(m, p.direction, last_good_delta)

        cur_pos = prev_pos + dxy
        segment_pred_local[int(tid)] = cur_pos

        if mode == 'model':
            last_good_delta = dxy.copy()

        step_diag_rows.append({
            'segment_idx': p.segment_idx,
            'step_idx': int(step_idx),
            'source_id': int(prev_id),
            'target_id': int(tid),
            'direction': p.direction,
            'predict_mode': mode,
            'matches': int(m.matches),
            'affine_inliers': int(m.affine_inliers),
            'sfm_inliers': int(m.sfm_inliers),
            'pred_dx': float(dxy[0]),
            'pred_dy': float(dxy[1]),
            'pred_x': float(cur_pos[0]),
            'pred_y': float(cur_pos[1]),
        })

        prev_id = int(tid)
        prev_pos = cur_pos

    closure_used = False
    closure_err = np.array([0.0, 0.0], dtype=np.float64)
    closure_inliers = 0

    if (p.closing_train_id is not None) and (len(p.ordered_test_ids) > 0):
        last_tid = int(p.ordered_test_ids[-1])
        close_tid = int(p.closing_train_id)
        m_close = estimate_pair_motion(last_tid, close_tid, SIFT_CFG)
        closure_inliers = int(m_close.affine_inliers)

        if m_close.success and (m_close.affine_inliers >= CLOSURE_MIN_INLIERS):
            delta_close = predict_delta(model, m_close)
            est_close = segment_pred_local[last_tid] + delta_close
            gt_close = train_pos_map[close_tid]
            closure_err = (gt_close - est_close).astype(np.float64)

            n = len(p.ordered_test_ids)
            for j, tid in enumerate(p.ordered_test_ids):
                alpha = float(j + 1) / float(max(1, n))
                segment_pred_local[int(tid)] = segment_pred_local[int(tid)] + alpha * closure_err

            closure_used = True

    for tid, pos in segment_pred_local.items():
        pred_map[int(tid)] = pos.astype(np.float64)

    segment_diag_rows.append({
        'segment_idx': p.segment_idx,
        'start_id': p.start_id,
        'end_id': p.end_id,
        'direction': p.direction,
        'anchor_train_id': p.anchor_train_id,
        'closing_train_id': p.closing_train_id,
        'closure_used': bool(closure_used),
        'closure_inliers': int(closure_inliers),
        'closure_err_x': float(closure_err[0]),
        'closure_err_y': float(closure_err[1]),
        'closure_err_norm': float(np.linalg.norm(closure_err)),
    })

# sanity
missing = [tid for tid in test_ids if int(tid) not in pred_map]
if len(missing) > 0:
    print('WARNING: missing predictions for ids:', missing[:20], '...')
    for tid in missing:
        prev_train = max([t for t in train_ids if t < int(tid)], default=min(train_ids))
        pred_map[int(tid)] = train_pos_map[int(prev_train)].copy()

segment_diag_df = pd.DataFrame(segment_diag_rows)
step_diag_df = pd.DataFrame(step_diag_rows)

print('predicted test frames:', len(pred_map), '/', len(test_ids))
display(segment_diag_df)
display(step_diag_df.head(40))


In [ ]:
# Build submission.csv
sub_rows = []
for tid in sorted(test_ids):
    p = pred_map[int(tid)]
    sub_rows.append({
        'id': int(tid),
        'x_pixel': float(p[0]),
        'y_pixel': float(p[1]),
    })
submission_df = pd.DataFrame(sub_rows).sort_values('id').reset_index(drop=True)

SUBMISSION_PATH.parent.mkdir(parents=True, exist_ok=True)
submission_df.to_csv(SUBMISSION_PATH, index=False)
submission_df.to_csv(ALT_SUBMISSION_PATH, index=False)

print('saved:', SUBMISSION_PATH)
print('saved:', ALT_SUBMISSION_PATH)
display(submission_df.head(20))
display(submission_df.tail(20))


In [ ]:
# Optional map overlay diagnostics (train GT + test prediction)
if MAP_PATH.exists():
    map_bgr = cv2.imread(str(MAP_PATH), cv2.IMREAD_COLOR)
    map_rgb = cv2.cvtColor(map_bgr, cv2.COLOR_BGR2RGB)

    train_xy = np.array([train_pos_map[i] for i in train_ids], dtype=np.float64)
    test_xy = submission_df[['x_pixel', 'y_pixel']].to_numpy(dtype=np.float64)

    plt.figure(figsize=(13, 9))
    plt.imshow(map_rgb)
    plt.scatter(train_xy[:, 0], train_xy[:, 1], s=8, c='deepskyblue', alpha=0.45, label='train GT')
    plt.scatter(test_xy[:, 0], test_xy[:, 1], s=14, c='red', alpha=0.8, label='test pred')
    plt.title('Notebook 15 | Train GT + Test predictions')
    plt.legend(loc='upper right')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print('map.png not found, skipping overlay.')


## Notes
- This notebook intentionally keeps the base SIFT/SfM pipeline from notebook 12 (pre feature-focus sweep).
- Segment planning follows your requested rule with the special reverse handling for IDs 1..12.
- Drift correction uses optional closure to the next available train anchor.
